# Import new noiseLAB data into main dataset

In [100]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from nptdms import TdmsFile
import imp
import seaborn as sns
import scipy.stats as st

from aeroacoustics import dataset as ds
from aeroacoustics import read_data_files as rd

In [ ]:
imp.reload(rd)
imp.reload(ds)

In [ ]:
%matplotlib

## Parameters

In [237]:
# root_dir_noiselab='/Volumes/Aeroacoustics/_ProcessedData/SliceData'
root_dir_noiselab='/Volumes/Aeroacoustics/NoiseLAB/DOE15AeroacousticsProject/Results/Tables'
root_dir_tdms='/Volumes/Tests/_raw data/Slow'

# list_filename='noiseLAB_files.p'
# dataset_file='aeroacoustics_dataset.p'

list_filename='/Volumes/Aeroacoustics/_ProcessedData/Dataset/noiseLAB_files.p'
list_filename_unweighted='/Volumes/Aeroacoustics/_ProcessedData/Dataset/noiseLAB_files_unweighted.p'
dataset_file='/Volumes/Aeroacoustics/_ProcessedData/Dataset/aeroacoustics_dataset.p'
dataset_file_unweighted='/Volumes/Aeroacoustics/_ProcessedData/Dataset/aeroacoustics_dataset_unweighted.p'
# dataset_file='aeroacoustics_dataset.p'

list_filename_fft='/Volumes/Aeroacoustics/_ProcessedData/Dataset/noiseLAB_files_fft.p'
list_filename_fft_unweighted='/Volumes/Aeroacoustics/_ProcessedData/Dataset/noiseLAB_files_fft_unweighted.p'
dataset_file_fft='/Volumes/Aeroacoustics/_ProcessedData/Dataset/aeroacoustics_dataset_fft.p'
dataset_file_fft_unweighted='/Volumes/Aeroacoustics/_ProcessedData/Dataset/aeroacoustics_dataset_fft_unweighted.p'
# dataset_file_fft='aeroacoustics_dataset_fft.p'

fig_dir = '/Volumes/Aeroacoustics/_ProcessedData/Figures'

mic_names = ['01N', '02N', '03N', '04C', '05C', '06S', '07S', '08S', '11N', '12C', '13S']

## Load data

In [ ]:
df_unw = pd.read_pickle(dataset_file_unweighted)
# df_Awt = pd.read_pickle(dataset_file)


In [ ]:
print(len(df_unw))
print(len(df_Awt))

In [ ]:
df_unw.tail()

In [ ]:
plt.figure()
# plt.plot(df_unw['EGD_OpCtl_TurbineStatus'])
# plt.plot(np.isnan(df_unw.WD_Nacelle_Mod))
# plt.plot(np.isnan(df_Awt.WD_Nacelle_Mod))
# plt.plot(df_Awt['Yaw_Offset_Cmd_Bin'])
# plt.plot(df_unw['Yaw_Offset_Cmd'])
plt.plot(df_unw['Yaw_Offset_Cmd_Bin'])
# plt.plot(df_unw['WD_Nacelle'])
# plt.plot(df_unw['WD_Nacelle_Mod'],'r')

In [247]:
df_Awt.columns.values

array(['01N_Leq', '02N_Leq', '03N_Leq', '04C_Leq', '05C_Leq', '06S_Leq',
       '07S_Leq', '08S_Leq', '11N_Leq', '12C_Leq', '13S_Leq',
       '01N_third_octave_spectrum', '02N_third_octave_spectrum',
       '03N_third_octave_spectrum', '04C_third_octave_spectrum',
       '05C_third_octave_spectrum', '06S_third_octave_spectrum',
       '07S_third_octave_spectrum', '08S_third_octave_spectrum',
       '11N_third_octave_spectrum', '12C_third_octave_spectrum',
       '13S_third_octave_spectrum', 'MS Excel Timestamp',
       'LabVIEW Timestamp', 'Scan Errors', 'Late Scans',
       'SCADA_nacelleWS', 'Hum1', 'Temp1', 'Hum2', 'Temp2',
       'Windspeed_38m', 'Windspeed_55m', 'Windspeed_87m', 'WD1_87m',
       'Air_Press_2', 'Air_Press_1', 'Wind_Direction_38m',
       'WindSpeed_80m', 'Precipitation', 'Tower Base Bend 1',
       'Tower Base Bend 2', 'Tower Base Torque', 'Active Power',
       'Reactive Power', 'Power Factor', 'Windspeed_AeroA_10m',
       'TowerTopBending_0', 'TowerTopBending_9

In [250]:

for mic in mic_names:
    df_unw.loc[~df_unw[mic+'_third_octave_spectrum'].isna(),mic+'_third_octave_spectrum_A'] = df_unw.loc[~df_unw[mic+'_third_octave_spectrum'].isna(),mic+'_third_octave_spectrum'].apply(lambda x: x + Aweight)
    df_unw.loc[~df_unw[mic+'_third_octave_spectrum'].isna(),mic+'_Leq'] = df_unw.loc[~df_unw[mic+'_third_octave_spectrum'].isna(),mic+'_third_octave_spectrum'].apply(energy_sum)
    df_unw.loc[~df_unw[mic+'_third_octave_spectrum_A'].isna(),mic+'_LAeq'] = df_unw.loc[~df_unw[mic+'_third_octave_spectrum_A'].isna(),mic+'_third_octave_spectrum_A'].apply(energy_sum)

In [ ]:
# define 1/3 octave band center frequencies
third_oct_center_freqs_unw = np.array([1, 1.2, 1.6, 2, 2.5, 3.2, 4, 5, 6.3, 8, 10, 12.5, 16, 20, 25, 31.5, 40, 50, 63, 80, 100, 125, 160, 200, 250, 315, 400, 500, 630, 800, 1000, 1250, 1600, 2000, 2500, 3150, 4000, 5000, 6300, 8000, 10000, 12500, 16000, 20000])
third_oct_center_freqs_Awt = np.array([20, 25, 31.5, 40, 50, 63, 80, 100, 125, 160, 200, 250, 315, 400, 500, 630, 800, 1000, 1250, 1600, 2000, 2500, 3150, 4000, 5000, 6300, 8000, 10000, 12500, 16000, 20000])

### Define A-weighting function to apply to unweighted spectra

In [220]:
freq2 = third_oct_center_freqs_unw**2;
freq4 = third_oct_center_freqs_unw**4;
Aweight = 10. * np.log10(1.562339 * freq4 / ((freq2 + 107.65265**2.) *
            (freq2 + 737.86223**2.))) / np.log10(10.) + 10. * np.log10(2.242881E+16 * freq4 / 
            ((freq2 + 20.598997**2.)**2. *
            (freq2 + 12194.22**2.)**2.)) / np.log10(10)

# R_A = ((12194**2)*freq4)/((freq2 + 20.6**2)*np.sqrt((freq2 + 107.7**2)*(freq2 + 737.9**2))*(freq2 + 12194**2))
# Alog = 20*np.log10(R_A) - 20*np.log10(R_A[np.where(center_freqsA == 1000)[0][0]])

plt.figure()
plt.plot(third_oct_center_freqs_unw,Aweight)

### Derive A weighted and Leq values

In [251]:
for mic in mic_names:
    df_unw.loc[~df_unw[mic+'_third_octave_spectrum'].isna(),mic+'_third_octave_spectrum_A'] = df_unw.loc[~df_unw[mic+'_third_octave_spectrum'].isna(),mic+'_third_octave_spectrum'].apply(lambda x: x + Aweight)
    df_unw.loc[~df_unw[mic+'_third_octave_spectrum'].isna(),mic+'_Leq'] = df_unw.loc[~df_unw[mic+'_third_octave_spectrum'].isna(),mic+'_third_octave_spectrum'].apply(energy_sum)
    df_unw.loc[~df_unw[mic+'_third_octave_spectrum_A'].isna(),mic+'_LAeq'] = df_unw.loc[~df_unw[mic+'_third_octave_spectrum_A'].isna(),mic+'_third_octave_spectrum_A'].apply(energy_sum)

### Correct yaw offset bins

In [277]:
df_unw.loc[(df_unw.index >= '2021-01-05 17:30:00') & (df_unw.index < '2021-01-05 17:55:00'),'Yaw_Offset_Cmd_Bin'] = 10.
df_unw.loc[(df_unw.index >= '2021-01-05 18:00:00') & (df_unw.index < '2021-01-05 18:26:00'),'Yaw_Offset_Cmd_Bin'] = 18.
df_unw.loc[(df_unw.index >= '2021-01-05 18:29:00') & (df_unw.index < '2021-01-05 18:55:00'),'Yaw_Offset_Cmd_Bin'] = 25.
df_unw.loc[(df_unw.index >= '2021-01-05 19:00:00') & (df_unw.index < '2021-01-05 19:20:00'),'Yaw_Offset_Cmd_Bin'] = -18.
df_unw.loc[(df_unw.index >= '2021-01-05 19:40:00') & (df_unw.index < '2021-01-05 19:50:00'),'Yaw_Offset_Cmd_Bin'] = 10.
df_unw.loc[(df_unw.index >= '2021-01-05 20:00:00') & (df_unw.index < '2021-01-05 20:20:00'),'Yaw_Offset_Cmd_Bin'] = 18.
df_unw.loc[(df_unw.index >= '2021-01-05 23:22:00') & (df_unw.index < '2021-01-05 23:26:30'),'Yaw_Offset_Cmd_Bin'] = -18.
df_unw.loc[(df_unw.index >= '2021-01-05 23:28:00') & (df_unw.index < '2021-01-05 23:31:00'),'Yaw_Offset_Cmd_Bin'] = 10.
df_unw.loc[(df_unw.index >= '2021-01-28 10:00:00') & (df_unw.index < '2021-01-28 13:25:00'),'Yaw_Offset_Cmd_Bin'] = 0.
df_unw.loc[(df_unw.index >= '2021-02-04 10:45:00') & (df_unw.index < '2021-02-04 11:05:00'),'Yaw_Offset_Cmd_Bin'] = 0.
df_unw.loc[(df_unw.index >= '2021-02-04 11:25:00') & (df_unw.index < '2021-02-04 11:39:00'),'Yaw_Offset_Cmd_Bin'] = 10.
df_unw.loc[(df_unw.index >= '2021-02-04 17:20:00') & (df_unw.index < '2021-02-04 17:35:00'),'Yaw_Offset_Cmd_Bin'] = 18.
df_unw.loc[(df_unw.index >= '2021-02-04 19:06:00') & (df_unw.index < '2021-02-04 19:30:00'),'Yaw_Offset_Cmd_Bin'] = 10.
df_unw.loc[(df_unw.index >= '2021-02-04 22:10:00') & (df_unw.index < '2021-02-04 22:30:00'),'Yaw_Offset_Cmd_Bin'] = 18.
df_unw.loc[(df_unw.index >= '2021-02-05 00:37:00') & (df_unw.index < '2021-02-05 01:05:00'),'Yaw_Offset_Cmd_Bin'] = 18.
df_unw.loc[(df_unw.index >= '2021-02-05 01:15:00') & (df_unw.index < '2021-02-05 01:30:00'),'Yaw_Offset_Cmd_Bin'] = 10.

df_Awt.loc[(df_Awt.index >= '2021-01-05 17:30:00') & (df_Awt.index < '2021-01-05 17:55:00'),'Yaw_Offset_Cmd_Bin'] = 10.
df_Awt.loc[(df_Awt.index >= '2021-01-05 18:00:00') & (df_Awt.index < '2021-01-05 18:26:00'),'Yaw_Offset_Cmd_Bin'] = 18.
df_Awt.loc[(df_Awt.index >= '2021-01-05 18:29:00') & (df_Awt.index < '2021-01-05 18:55:00'),'Yaw_Offset_Cmd_Bin'] = 25.
df_Awt.loc[(df_Awt.index >= '2021-01-05 19:00:00') & (df_Awt.index < '2021-01-05 19:20:00'),'Yaw_Offset_Cmd_Bin'] = -18.
df_Awt.loc[(df_Awt.index >= '2021-01-05 19:40:00') & (df_Awt.index < '2021-01-05 19:50:00'),'Yaw_Offset_Cmd_Bin'] = 10.
df_Awt.loc[(df_Awt.index >= '2021-01-05 20:00:00') & (df_Awt.index < '2021-01-05 20:20:00'),'Yaw_Offset_Cmd_Bin'] = 18.
df_Awt.loc[(df_Awt.index >= '2021-01-05 23:22:00') & (df_Awt.index < '2021-01-05 23:26:30'),'Yaw_Offset_Cmd_Bin'] = -18.
df_Awt.loc[(df_Awt.index >= '2021-01-05 23:28:00') & (df_Awt.index < '2021-01-05 23:31:00'),'Yaw_Offset_Cmd_Bin'] = 10.
df_Awt.loc[(df_Awt.index >= '2021-01-28 10:00:00') & (df_Awt.index < '2021-01-28 13:25:00'),'Yaw_Offset_Cmd_Bin'] = 0.
df_Awt.loc[(df_Awt.index >= '2021-02-04 10:45:00') & (df_Awt.index < '2021-02-04 11:05:00'),'Yaw_Offset_Cmd_Bin'] = 0.
df_Awt.loc[(df_Awt.index >= '2021-02-04 11:25:00') & (df_Awt.index < '2021-02-04 11:39:00'),'Yaw_Offset_Cmd_Bin'] = 10.

### Filter data

In [278]:
# use_Awt = True
use_Awt = False

use_any_WD_background = False

if use_Awt:
    df_filt = df_Awt.copy()
else:
    df_filt = df_unw.copy()

print(len(df_filt))

# TEMP!
# df_filt['Yaw_Offset_Cmd'] = df_filt.WD_Nacelle - df_filt.WD_Nacelle_Mod

# df_filt['Yaw_Offset_Cmd_Bin'] = 0.
# df_filt.loc[df_filt['Yaw_Offset_Cmd'] < -9.,'Yaw_Offset_Cmd_Bin'] = -18.
# df_filt.loc[(df_filt['Yaw_Offset_Cmd'] >= 5.) & (df_filt['Yaw_Offset_Cmd'] < 14.),'Yaw_Offset_Cmd_Bin'] = 10.
# df_filt.loc[(df_filt['Yaw_Offset_Cmd'] >= 14.) & (df_filt['Yaw_Offset_Cmd'] < 21.5),'Yaw_Offset_Cmd_Bin'] = 18.
# df_filt.loc[df_filt['Yaw_Offset_Cmd'] >= 21.5,'Yaw_Offset_Cmd_Bin'] = 25.

# plt.figure()
# plt.plot(df_filt['Active Power'])

# get rid of spurious offset commands
# df_filt.loc[df_filt.index < '2021-01-01','Yaw_Offset_Cmd_Bin'] = 0.

# filter to remove 5 minutes after a yaw offset command change
change_points = df_filt.loc[df_filt['Yaw_Offset_Cmd_Bin'].diff().abs() > 0].index
for d in change_points:
    df_filt = df_filt[(df_filt.index < d) | (df_filt.index >= d + pd.Timedelta('5 min'))]

print(len(df_filt))
    
# create derived signals
df_filt['Turbine Operating'] = (df_filt['EGD_OpCtl_TurbineStatus'] == 2)

# catch operation based on power too, and remove from dataset
df_filt = df_filt[(df_filt['Active Power'] < 10) | df_filt['Turbine Operating']]
print(len(df_filt))
# This should catch cases where the yaw offset command is NaN beause the turbine is not running
df_filt.loc[~df_filt['Turbine Operating'],'Yaw_Offset_Cmd_Bin'] = 0.
df_filt['Yaw_Position_Normal'] = (df_filt['Yaw_Encoder'] + df_filt['Yaw_Offset_Cmd_Bin']) % 360.

# plt.plot(df_filt['Active Power'])

# filter to yaw position +/- 15 degrees from 285
df_filt['Direction Valid'] = (df_filt['Yaw_Position_Normal'] >= 270.) & (df_filt['Yaw_Position_Normal'] <= 300.)

if use_any_WD_background:
    df_filt = df_filt[df_filt['Direction Valid'] | ~df_filt['Turbine Operating']]
else:
    df_filt = df_filt[df_filt['Direction Valid']]
    
print(len(df_filt))
# plt.plot(df_filt['Active Power'],'r')

df_filt['WindSpeed_80m_round'] = df_filt['WindSpeed_80m'].round()

# filter to rows when all mics are available
cols2check = ['01N_Leq', '02N_Leq', '03N_Leq', '04C_Leq', '05C_Leq', '06S_Leq', '07S_Leq', '08S_Leq', '11N_Leq', '12C_Leq', '13S_Leq']
# df_filt = df_filt.dropna(subset=cols2check)
print(len(df_filt))

16820
14261
13162
7546
7546


In [ ]:
# plt.figure()
# plt.plot(df_filt['Temp1'])
df_filtA = df_filt.copy()

## Plot 1/3 octave band spectra by wind speed for a given mic and offset

In [226]:
def energy_avg(values):
    return 10*np.log10(np.mean(10**(values/10)))

def energy_sum(values):
    return 10*np.log10(np.sum(10**(values/10)))

def energy_subtract(values1,values2):
    return 10*np.log10(10**(values1/10) - 10**(values2/10))

def energy_avg_sem(values,stdev=False):
    Lmean =  10*np.log10(np.mean(10**(values.dropna()/10)))
    
    if isinstance(values.dropna()[0],float):
        if stdev:
            return np.sqrt(np.sum((values.dropna()-Lmean)**2)/(len(values.dropna())))
        else:
            return np.sqrt(np.sum((values.dropna()-Lmean)**2)/((len(values.dropna())-1)*len(values.dropna())))
    else:
        values_array = pd.DataFrame(values.dropna().tolist()).to_numpy()
        if stdev:
            return np.sqrt(np.sum((values_array-Lmean)**2,0)/(len(values_array)))
        else:
            return np.sqrt(np.sum((values_array-Lmean)**2,0)/((len(values_array)-1)*len(values_array)))
    
def energy_avg_corrected_sem(Leq_operating,Leq_operating_sem,Leq_background,Leq_background_sem):
    if isinstance(Leq_operating,float):
        if Leq_operating - Leq_background > 3.:
            return np.sqrt((Leq_operating_sem*10**(Leq_operating/10))**2 + (Leq_background_sem*10**(Leq_background/10))**2)/(10**(Leq_operating/10) - 10**(Leq_background/10))
        else:
            return np.sqrt((Leq_operating_sem*10**(Leq_operating/10))**2 + (Leq_background_sem*10**((Leq_operating-3)/10))**2)/(10**(Leq_operating/10) - 10**((Leq_operating-3)/10))
    else:    
        corrected_sem = np.zeros(len(Leq_operating))
        for i in range(len(Leq_operating)):
            if Leq_operating[i] - Leq_background[i] > 3.:
                corrected_sem[i] = np.sqrt((Leq_operating_sem[i]*10**(Leq_operating[i]/10))**2 + (Leq_background_sem[i]*10**(Leq_background[i]/10))**2)/(10**(Leq_operating[i]/10) - 10**(Leq_background[i]/10))
            else:
                corrected_sem[i] = np.sqrt((Leq_operating_sem[i]*10**(Leq_operating[i]/10))**2 + (Leq_background_sem[i]*10**((Leq_operating[i]-3)/10))**2)/(10**(Leq_operating[i]/10) - 10**((Leq_operating[i]-3)/10))
        return corrected_sem


In [174]:
values = df_filt.loc[(df_filt['Yaw_Offset_Cmd_Bin'] == yaw_offset_target) & df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+'_Leq']
# Lmean =  10*np.log10(np.mean(10**(values/10)))
# np.sqrt(np.sum((values.dropna()-Lmean)**2)/((len(values.dropna())-1)*len(values.dropna())))
# print(values.dropna())
# values = df_filt.loc[(df_filt['Yaw_Offset_Cmd_Bin'] == yaw_offset_target) & df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+'_third_octave_spectrum']
# Lmean =  10*np.log10(np.mean(10**(values.dropna()/10)))
# values_array = pd.DataFrame(values.dropna().tolist()).to_numpy()
# np.sqrt(np.sum((values_array-Lmean)**2,0)/((len(values_array)-1)*len(values_array)))
energy_avg_sem(values)

0.37413251079121496

In [271]:
df_filt['03N_third_octave_spectrum'].isna()


time
2020-12-01 09:56:38    False
2020-12-01 09:56:48    False
2020-12-01 09:56:58    False
2020-12-01 09:57:08    False
2020-12-01 09:57:18    False
                       ...  
2021-02-05 02:11:28     True
2021-02-05 02:11:35    False
2021-02-05 02:11:38     True
2021-02-05 02:11:45    False
2021-02-05 02:11:48     True
Name: 03N_third_octave_spectrum, Length: 7546, dtype: bool

In [296]:
save_figs = False
ws_col = 'WindSpeed_80m_round'

use_Awt = True

stdev=False

# freq_bounds = None
freq_bounds = [500,2000]

yaw_offset_target = 0.

mic = '04C'

ws_list = np.arange(6.,15.,1.)

df_results = plot_third_oct_SPL_multi_ws(df_filt=df_filt,mic=mic,ws_list=ws_list,yaw_offset_target = yaw_offset_target,ws_col=ws_col,save_figs=save_figs,stdev=stdev,center_freqs=third_oct_center_freqs_unw,use_Awt=use_Awt,freq_bounds=freq_bounds)

/Users/esimley/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log10
  
/Users/esimley/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log10
  
/Users/esimley/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log10
  
/Users/esimley/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log10
  
/Users/esimley/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log10
  
/Users/esimley/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log10
  
/Users/esimley/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log10
  
/Users/esimley/opt/anaconda3/lib/python3.7/site-

In [301]:
save_figs = False
ws_col = 'WindSpeed_80m_round'

stdev=False

yaw_offset_targets = [-18.,0.,10.,18.,25.]

use_Awt = True

# freq_bounds = None
freq_bounds = [500,2000]

mic = '04C'

ws_list = np.arange(6.,15.,1.)

ws = 10.

# if use_Awt:
#     center_freqs = third_oct_center_freqs_Awt
# else:
#     center_freqs = third_oct_center_freqs_unw

df_results = plot_third_oct_SPL_multi_offset(df_filt,mic,ws,yaw_offset_targets,ws_col=ws_col,save_figs=save_figs,stdev=stdev,center_freqs = third_oct_center_freqs_unw,use_Awt=use_Awt,freq_bounds = freq_bounds)

/Users/esimley/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log10
  
/Users/esimley/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log10
  
/Users/esimley/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log10
  
/Users/esimley/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log10
  
/Users/esimley/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log10
  
/Users/esimley/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log10
  
/Users/esimley/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log10
  
/Users/esimley/opt/anaconda3/lib/python3.7/site-

In [ ]:
# plt.figure()
# plt.scatter(df_filt.loc[~df_filt['Turbine Operating'],ws_col],df_filt.loc[~df_filt['Turbine Operating'],'Active Power'])
# plt.figure()
# plt.plot(df_filt.loc[~np.isnan(df_filt['01N_Leq']),'Yaw_Offset_Cmd_Bin'])
# plt.plot(df_filt['Yaw_Offset_Cmd_Bin'])

In [295]:


def plot_third_oct_SPL_multi_ws(df_filt,mic,ws_list,yaw_offset_target = 0.,ws_col='WindSpeed_80m_round',save_figs=False,stdev=False,center_freqs = third_oct_center_freqs_unw,use_Awt=False,freq_bounds=None):
    
    # first plot Leq vs wind speed
    fig = plt.figure()

    if use_Awt:
        Leq_var = '_LAeq'
        third_oct_var = '_third_octave_spectrum_A'
    else:
        Leq_var = '_Leq'
        third_oct_var = '_third_octave_spectrum'

    df_results = pd.DataFrame()
        
    Leq_operating = np.zeros(len(ws_list))
    Leq_background = np.zeros(len(ws_list))
    Leq_operating_sem = np.zeros(len(ws_list))
    Leq_background_sem = np.zeros(len(ws_list))
    Leq_corrected_sem = np.zeros(len(ws_list))
    
    
    if freq_bounds is not None:
        i_st = np.where(center_freqs==freq_bounds[0])[0][0]
        i_en = np.where(center_freqs==freq_bounds[1])[0][0]
        
        for i_ws, ws in enumerate(ws_list):
            Leq_operating[i_ws] = energy_avg(df_filt.loc[(df_filt['Yaw_Offset_Cmd_Bin'] == yaw_offset_target) & df_filt['Turbine Operating'] & (df_filt[ws_col] == ws) & ~df_filt[mic+third_oct_var].isna(),mic+third_oct_var].apply(lambda x: energy_sum(x[i_st:i_en+1])))
            Leq_background[i_ws] = energy_avg(df_filt.loc[~df_filt['Turbine Operating'] & (df_filt[ws_col] == ws) & ~df_filt[mic+third_oct_var].isna(),mic+third_oct_var].apply(lambda x: energy_sum(x[i_st:i_en+1])))
            Leq_operating_sem[i_ws] = energy_avg_sem(df_filt.loc[(df_filt['Yaw_Offset_Cmd_Bin'] == yaw_offset_target) & df_filt['Turbine Operating'] & (df_filt[ws_col] == ws) & ~df_filt[mic+third_oct_var].isna(),mic+third_oct_var].apply(lambda x: energy_sum(x[i_st:i_en+1])),stdev=stdev)
            Leq_background_sem[i_ws] = energy_avg_sem(df_filt.loc[~df_filt['Turbine Operating'] & (df_filt[ws_col] == ws) & ~df_filt[mic+third_oct_var].isna(),mic+third_oct_var].apply(lambda x: energy_sum(x[i_st:i_en+1])),stdev=stdev)
            Leq_corrected_sem[i_ws] = energy_avg_corrected_sem(Leq_operating[i_ws],Leq_operating_sem[i_ws],Leq_background[i_ws],Leq_background_sem[i_ws])
    else:
        for i_ws, ws in enumerate(ws_list):
            Leq_operating[i_ws] = energy_avg(df_filt.loc[(df_filt['Yaw_Offset_Cmd_Bin'] == yaw_offset_target) & df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+Leq_var])
            Leq_background[i_ws] = energy_avg(df_filt.loc[~df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+Leq_var])
            Leq_operating_sem[i_ws] = energy_avg_sem(df_filt.loc[(df_filt['Yaw_Offset_Cmd_Bin'] == yaw_offset_target) & df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+Leq_var],stdev=stdev)
            Leq_background_sem[i_ws] = energy_avg_sem(df_filt.loc[~df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+Leq_var],stdev=stdev)
            Leq_corrected_sem[i_ws] = energy_avg_corrected_sem(Leq_operating[i_ws],Leq_operating_sem[i_ws],Leq_background[i_ws],Leq_background_sem[i_ws])

    for i_ws, ws in enumerate(ws_list):
        df_results = df_results.append(pd.DataFrame(
                   {'ws':[ws],'Leq_operating':[Leq_operating[i_ws]],
                   'Leq_background':[Leq_background[i_ws]],
                   'Leq_corrected':[energy_subtract(Leq_operating[i_ws],Leq_background[i_ws])],
                   'Leq_operating_sem':[Leq_operating_sem[i_ws]],
                   'Leq_background_sem':[Leq_background_sem[i_ws]],
                   'Leq_corrected_sem':[Leq_corrected_sem[i_ws]]}))
            
    plt.plot(ws_list,Leq_operating,color='g',linewidth=2.,label='Operating')
    plt.fill_between(ws_list,Leq_operating-Leq_operating_sem,Leq_operating+Leq_operating_sem,alpha=0.3,color='g',label="_nolegend_")
    plt.plot(ws_list,Leq_background,color='b',linewidth=2.,label='Background')
    plt.fill_between(ws_list,Leq_background-Leq_background_sem,Leq_background+Leq_background_sem,alpha=0.3,color='b',label="_nolegend_")
    plt.plot(ws_list,energy_subtract(Leq_operating,Leq_background),color='r',linewidth=2.,label='Corrected')
    plt.fill_between(ws_list,energy_subtract(Leq_operating,Leq_background)-Leq_corrected_sem,energy_subtract(Leq_operating,Leq_background)+Leq_corrected_sem,alpha=0.3,color='r',label="_nolegend_")

    plt.xlabel('Wind Speed (m/s)')

    if use_Awt:
        plt.ylabel('$L_{eq}$ (dBA)')
    else:
        plt.ylabel('$L_{eq}$ (dB)')

    plt.grid()
    plt.title('Mic '+mic+', Yaw Offset = '+str(yaw_offset_target)+'$^\circ$')
    plt.legend()
    plt.xlim([ws_list[0],ws_list[-1]])

    if save_figs:    
        if use_Awt:
            fig.savefig(os.path.join(fig_dir,'LAeq_'+mic+'_'+str(yaw_offset_target)+'degOffset.pdf'), bbox_inches='tight')
        else:
            fig.savefig(os.path.join(fig_dir,'Leq_'+mic+'_'+str(yaw_offset_target)+'degOffset.pdf'), bbox_inches='tight')

    fig, axs = plt.subplots(3,4,sharex=True,sharey=True,figsize=(14,9))

    ws_list = np.arange(6.,15.,1.)

    df_thirdoct = pd.DataFrame()
    
    for i_ws, ws in enumerate(ws_list):

        Leq_operating = energy_avg(df_filt.loc[(df_filt['Yaw_Offset_Cmd_Bin'] == yaw_offset_target) & df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+third_oct_var])
        Leq_background = energy_avg(df_filt.loc[~df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+third_oct_var])

        if (len(df_filt.loc[(df_filt['Yaw_Offset_Cmd_Bin'] == yaw_offset_target) & df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+third_oct_var]) > 0) & (len(df_filt.loc[~df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+third_oct_var]) > 0):
            axs[i_ws % 3][int(np.floor(i_ws/3))].plot(center_freqs,Leq_operating,color='g',linewidth=1.5,label='Operating')
            Leq_operating_sem = energy_avg_sem(df_filt.loc[(df_filt['Yaw_Offset_Cmd_Bin'] == yaw_offset_target) & df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+third_oct_var],stdev=stdev)
            axs[i_ws % 3][int(np.floor(i_ws/3))].fill_between(center_freqs,Leq_operating-Leq_operating_sem,Leq_operating+Leq_operating_sem,alpha=0.3,color='g',label="_nolegend_")
            

        if len(df_filt.loc[~df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+third_oct_var]) > 0:
            axs[i_ws % 3][int(np.floor(i_ws/3))].plot(center_freqs,Leq_background,color='b',linewidth=1.5,label='Background')
            Leq_background_sem = energy_avg_sem(df_filt.loc[~df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+third_oct_var],stdev=stdev)
            axs[i_ws % 3][int(np.floor(i_ws/3))].fill_between(center_freqs,Leq_background-Leq_background_sem,Leq_background+Leq_background_sem,alpha=0.3,color='b',label="_nolegend_")
            axs[i_ws % 3][int(np.floor(i_ws/3))].plot(center_freqs,energy_subtract(Leq_operating,Leq_background),color='r',linewidth=1.5,label='Corrected')
            Leq_corrected_sem = energy_avg_corrected_sem(Leq_operating,Leq_operating_sem,Leq_background,Leq_background_sem)
            axs[i_ws % 3][int(np.floor(i_ws/3))].fill_between(center_freqs,energy_subtract(Leq_operating,Leq_background)-Leq_corrected_sem,energy_subtract(Leq_operating,Leq_background)+Leq_corrected_sem,alpha=0.3,color='r',label="_nolegend_")
        axs[i_ws % 3][int(np.floor(i_ws/3))].legend()
        axs[i_ws % 3][int(np.floor(i_ws/3))].grid()
        axs[i_ws % 3][int(np.floor(i_ws/3))].set_title('Mic '+mic+', Yaw Offset = '+str(yaw_offset_target)+'$^\circ$, '+str(ws)+' m/s')
        axs[i_ws % 3][int(np.floor(i_ws/3))].set_xscale('log')
        
        df_thirdoct = df_thirdoct.append(pd.DataFrame(
                   {'ws':[ws],'Leq_operating_thirdoct':[Leq_operating],
                   'Leq_background_thirdoct':[Leq_background],
                   'Leq_corrected_thirdoct':[energy_subtract(Leq_operating,Leq_background)],
                   'Leq_operating_sem_thirdoct':[Leq_operating_sem],
                   'Leq_background_sem_thirdoct':[Leq_background_sem],
                   'Leq_corrected_sem_thirdoct':[Leq_corrected_sem]}))

    df_results = pd.merge(df_results,df_thirdoct,on='ws')
        
    for i in range(3):
        if use_Awt:
            axs[i][0].set_ylabel('$L_{eq}$ (dBA)')
        else:
            axs[i][0].set_ylabel('$L_{eq}$ (dB)')

    for i in range(4):
        axs[2][i].set_xlabel('Frequency (Hz)')

    if use_Awt:
        axs[0][0].set_xlim([20,10000])
    else:
        axs[0][0].set_xlim([1,10000])

    plt.tight_layout()

    if save_figs:
        if use_Awt:
            fig.savefig(os.path.join(fig_dir,'ThirdOctave_LAeq_'+mic+'_'+str(yaw_offset_target)+'degOffset.pdf'), bbox_inches='tight')
        else:
            fig.savefig(os.path.join(fig_dir,'ThirdOctave_Leq_'+mic+'_'+str(yaw_offset_target)+'degOffset.pdf'), bbox_inches='tight')
            
    return df_results

## Plot 1/3 octave band spectra by yaw offset

In [300]:


def plot_third_oct_SPL_multi_offset(df_filt,mic,ws,yaw_offset_targets = [-18.,0.,10.,18.,25.],ws_col='WindSpeed_80m_round',save_figs=False,stdev=False,center_freqs = third_oct_center_freqs_unw,use_Awt=False,freq_bounds=None):

    color_list = ['#4477AA','#228833','#EE6677','#CCBB44','#AA3377']
    
    if use_Awt:
        Leq_var = '_LAeq'
        third_oct_var = '_third_octave_spectrum_A'
    else:
        Leq_var = '_Leq'
        third_oct_var = '_third_octave_spectrum'
    
    # first plot Leq vs wind speed
    fig = plt.figure()

    df_results = pd.DataFrame()
    
    Leq_operating = np.zeros(len(yaw_offset_targets))
    Leq_background = np.zeros(len(yaw_offset_targets))
    Leq_operating_sem = np.zeros(len(yaw_offset_targets))
    Leq_background_sem = np.zeros(len(yaw_offset_targets))
    Leq_corrected_sem = np.zeros(len(yaw_offset_targets))
        
    if freq_bounds is not None:
        i_st = np.where(center_freqs==freq_bounds[0])[0][0]
        i_en = np.where(center_freqs==freq_bounds[1])[0][0]
        
        for i_offst, offst in enumerate(yaw_offset_targets):
            Leq_operating[i_offst] = energy_avg(df_filt.loc[(df_filt['Yaw_Offset_Cmd_Bin'] == offst) & df_filt['Turbine Operating'] & (df_filt[ws_col] == ws) & ~df_filt[mic+third_oct_var].isna(),mic+third_oct_var].apply(lambda x: energy_sum(x[i_st:i_en+1])))
            Leq_background[i_offst] = energy_avg(df_filt.loc[~df_filt['Turbine Operating'] & (df_filt[ws_col] == ws) & ~df_filt[mic+third_oct_var].isna(),mic+third_oct_var].apply(lambda x: energy_sum(x[i_st:i_en+1])))
            Leq_operating_sem[i_offst] = energy_avg_sem(df_filt.loc[(df_filt['Yaw_Offset_Cmd_Bin'] == offst) & df_filt['Turbine Operating'] & (df_filt[ws_col] == ws) & ~df_filt[mic+third_oct_var].isna(),mic+third_oct_var].apply(lambda x: energy_sum(x[i_st:i_en+1])),stdev=stdev)
            Leq_background_sem[i_offst] = energy_avg_sem(df_filt.loc[~df_filt['Turbine Operating'] & (df_filt[ws_col] == ws) & ~df_filt[mic+third_oct_var].isna(),mic+third_oct_var].apply(lambda x: energy_sum(x[i_st:i_en+1])),stdev=stdev)
            Leq_corrected_sem[i_offst] = energy_avg_corrected_sem(Leq_operating[i_offst],Leq_operating_sem[i_offst],Leq_background[i_offst],Leq_background_sem[i_offst])
    else:
        for i_offst, offst in enumerate(yaw_offset_targets):
            Leq_operating[i_offst] = energy_avg(df_filt.loc[(df_filt['Yaw_Offset_Cmd_Bin'] == offst) & df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+Leq_var])
            Leq_background[i_offst] = energy_avg(df_filt.loc[~df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+Leq_var])
            Leq_operating_sem[i_offst] = energy_avg_sem(df_filt.loc[(df_filt['Yaw_Offset_Cmd_Bin'] == offst) & df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+Leq_var],stdev=stdev)
            Leq_background_sem[i_offst] = energy_avg_sem(df_filt.loc[~df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+Leq_var],stdev=stdev)
            Leq_corrected_sem[i_offst] = energy_avg_corrected_sem(Leq_operating[i_offst],Leq_operating_sem[i_offst],Leq_background[i_offst],Leq_background_sem[i_offst])
    
    for i_offst, offst in enumerate(yaw_offset_targets):
        df_results = df_results.append(pd.DataFrame(
                   {'offst':[offst],'Leq_operating':[Leq_operating[i_offst]],
                   'Leq_background':[Leq_background[i_offst]],
                   'Leq_corrected':[energy_subtract(Leq_operating[i_offst],Leq_background[i_offst])],
                   'Leq_operating_sem':[Leq_operating_sem[i_offst]],
                   'Leq_background_sem':[Leq_background_sem[i_offst]],
                   'Leq_corrected_sem':[Leq_corrected_sem[i_offst]]}))
    
    plt.plot(yaw_offset_targets,Leq_operating,color='g',linewidth=2.,label='Operating')
    plt.fill_between(yaw_offset_targets,Leq_operating-Leq_operating_sem,Leq_operating+Leq_operating_sem,alpha=0.3,color='g',label="_nolegend_")
    plt.plot(yaw_offset_targets,Leq_background,color='b',linewidth=2.,label='Background')
    plt.fill_between(yaw_offset_targets,Leq_background-Leq_background_sem,Leq_background+Leq_background_sem,alpha=0.3,color='b',label="_nolegend_")
    plt.plot(yaw_offset_targets,energy_subtract(Leq_operating,Leq_background),color='r',linewidth=2.,label='Corrected')
    plt.fill_between(yaw_offset_targets,energy_subtract(Leq_operating,Leq_background)-Leq_corrected_sem,energy_subtract(Leq_operating,Leq_background)+Leq_corrected_sem,alpha=0.3,color='r',label="_nolegend_")    
    
    plt.xlabel('Yaw Offset Target (deg)')

    if use_Awt:
        plt.ylabel('$L_{eq}$ (dBA)')
    else:
        plt.ylabel('$L_{eq}$ (dB)')

    plt.grid()
    plt.title('Mic '+mic+', Wind Speed = '+str(ws)+' m/s')
    plt.legend()
    plt.xlim([np.min(yaw_offset_targets),np.max(yaw_offset_targets)])

    if save_figs:    
        if use_Awt:
            fig.savefig(os.path.join(fig_dir,'LAeq_'+mic+'_'+str(ws)+'ms.pdf'), bbox_inches='tight')
        else:
            fig.savefig(os.path.join(fig_dir,'Leq_'+mic+'_'+str(ws)+'ms.pdf'), bbox_inches='tight')

    fig, axs = plt.subplots(2,3,sharex=True,sharey=True,figsize=(14,9))

    fig1, axs1 = plt.subplots(1,1,figsize=(6,4))

    df_thirdoct = pd.DataFrame()
    
    for i_offst, offst in enumerate(yaw_offset_targets):

        Leq_operating = energy_avg(df_filt.loc[(df_filt['Yaw_Offset_Cmd_Bin'] == offst) & df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+third_oct_var])
        Leq_background = energy_avg(df_filt.loc[~df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+third_oct_var])

        if len(df_filt.loc[(df_filt['Yaw_Offset_Cmd_Bin'] == offst) & df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+third_oct_var]) > 0:
            axs[i_offst % 2][int(np.floor(i_offst/2))].plot(center_freqs,Leq_operating,color='g',linewidth=1.5,label='Operating')
            Leq_operating_sem = energy_avg_sem(df_filt.loc[(df_filt['Yaw_Offset_Cmd_Bin'] == offst) & df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+third_oct_var],stdev=stdev)
            axs[i_offst % 2][int(np.floor(i_offst/2))].fill_between(center_freqs,Leq_operating-Leq_operating_sem,Leq_operating+Leq_operating_sem,alpha=0.3,color='g',label="_nolegend_")
            
        if len(df_filt.loc[~df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+third_oct_var]) > 0:
            axs[i_offst % 2][int(np.floor(i_offst/2))].plot(center_freqs,Leq_background,color='b',linewidth=1.5,label='Background')
            axs[i_offst % 2][int(np.floor(i_offst/2))].plot(center_freqs,energy_subtract(Leq_operating,Leq_background),color='r',linewidth=1.5,label='Corrected')
            
            Leq_background_sem = energy_avg_sem(df_filt.loc[~df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+third_oct_var],stdev=stdev)
            axs[i_offst % 2][int(np.floor(i_offst/2))].fill_between(center_freqs,Leq_background-Leq_background_sem,Leq_background+Leq_background_sem,alpha=0.3,color='b',label="_nolegend_")
            Leq_corrected_sem = energy_avg_corrected_sem(Leq_operating,Leq_operating_sem,Leq_background,Leq_background_sem)
            axs[i_offst % 2][int(np.floor(i_offst/2))].fill_between(center_freqs,energy_subtract(Leq_operating,Leq_background)-Leq_corrected_sem,energy_subtract(Leq_operating,Leq_background)+Leq_corrected_sem,alpha=0.3,color='r',label="_nolegend_")  
            
        axs[i_offst % 2][int(np.floor(i_offst/2))].legend()
        axs[i_offst % 2][int(np.floor(i_offst/2))].grid()
        axs[i_offst % 2][int(np.floor(i_offst/2))].set_title('Mic '+mic+', Yaw Offset = '+str(offst)+'$^\circ$, '+str(ws)+' m/s')
        axs[i_offst % 2][int(np.floor(i_offst/2))].set_xscale('log')

        axs1.plot(center_freqs,energy_subtract(Leq_operating,Leq_background),color=color_list[i_offst],linewidth=2.,label='Offset = '+str(offst)+' deg')
        axs1.fill_between(center_freqs,energy_subtract(Leq_operating,Leq_background)-Leq_corrected_sem,energy_subtract(Leq_operating,Leq_background)+Leq_corrected_sem,alpha=0.3,color=color_list[i_offst],label="_nolegend_")  
        
        df_thirdoct = df_thirdoct.append(pd.DataFrame(
                   {'offst':[offst],'Leq_operating_thirdoct':[Leq_operating],
                   'Leq_background_thirdoct':[Leq_background],
                   'Leq_corrected_thirdoct':[energy_subtract(Leq_operating,Leq_background)],
                   'Leq_operating_sem_thirdoct':[Leq_operating_sem],
                   'Leq_background_sem_thirdoct':[Leq_background_sem],
                   'Leq_corrected_sem_thirdoct':[Leq_corrected_sem]}))

    df_results = pd.merge(df_results,df_thirdoct,on='offst')
        
    for i in range(2):
        if use_Awt:
            axs[i][0].set_ylabel('$L_{eq}$ (dBA)')
        else:
            axs[i][0].set_ylabel('$L_{eq}$ (dB)')

    for i in range(3):
        axs[1][i].set_xlabel('Frequency (Hz)')

    if use_Awt:
        axs[0][0].set_xlim([20,10000])
    else:
        axs[0][0].set_xlim([1,10000])

    axs1.legend()
    axs1.grid()
    axs1.set_title('Background Corrected Noise, Mic '+mic+', '+str(ws)+' m/s')
    axs1.set_xscale('log')
    axs1.set_xlabel('Frequency (Hz)')

    if use_Awt:
        axs1.set_ylabel('$L_{eq}$ (dBA)')
        axs1.set_xlim([20,10000])
    else:
        axs1.set_ylabel('$L_{eq}$ (dB)')
        axs1.set_xlim([200,10000])

    plt.tight_layout()

    if save_figs:
        if use_Awt:
            fig.savefig(os.path.join(fig_dir,'ThirdOctave_LAeq_'+mic+'_'+str(ws)+'ms.pdf'), bbox_inches='tight')
            fig1.savefig(os.path.join(fig_dir,'ThirdOctave_1Plot_LAeq_'+mic+'_'+str(ws)+'ms.pdf'), bbox_inches='tight')
        else:
            fig.savefig(os.path.join(fig_dir,'ThirdOctave_Leq_'+mic+'_'+str(ws)+'ms.pdf'), bbox_inches='tight')
            fig1.savefig(os.path.join(fig_dir,'ThirdOctave_1Plot_Leq_'+mic+'_'+str(ws)+'ms.pdf'), bbox_inches='tight')
            
    return df_results

In [ ]:
mic = '04C'
ws = 10.
yaw_offset_target = 10.
df_filt['WindSpeed_80m_round'] = df_filt['WindSpeed_80m'].round()
df_filtA['WindSpeed_80m_round'] = df_filtA['WindSpeed_80m'].round()
Leq_operating = energy_avg(df_filt.loc[(df_filt['Yaw_Offset_Cmd_Bin'] == yaw_offset_target) & df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+'_third_octave_spectrum'])
LeqA_operating = energy_avg(df_filtA.loc[(df_filtA['Yaw_Offset_Cmd_Bin'] == yaw_offset_target) & df_filtA['Turbine Operating'] & (df_filtA[ws_col] == ws),mic+'_third_octave_spectrum'])


center_freqsA = third_oct_center_freqs_Awt
center_freqs = third_oct_center_freqs_unw

plt.figure()
plt.plot(center_freqsA,LeqA_operating - Alog,color='r',linewidth=3.,label='Corrected A-weighted Spectrum')
plt.plot(center_freqs,Leq_operating,color='b',linewidth=1.5,label='Unweighted Spectrum')
plt.plot(center_freqsA,LeqA_operating,color='g',linewidth=1.5,label='A-weighted Spectrum')

plt.xscale('log')
plt.xlabel('Frequency (Hz)')
plt.ylabel('dB')
plt.legend()
plt.grid()
plt.title('Mic 04C, 10 m/s, 10 deg yaw offset')

In [ ]:
# ~np.isnan(Leq_background)
len(df_filt.loc[(df_filt['Yaw_Offset_Cmd_Bin'] == yaw_offset_target) & ~df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+'_third_octave_spectrum'])

In [ ]:
energy_avg(df_filt.loc[(df_filt['Yaw_Offset_Cmd_Bin'] == yaw_offset_target) & df_filt['Turbine Operating'] & (df_filt[ws_col] == ws),mic+'_third_octave_spectrum'])

In [ ]:
plt.figure()
plt.scatter(df_filt.loc[~df_filt['Turbine Operating'],'NacelleWindSpeed'],df_filt.loc[~df_filt['Turbine Operating'],'Active Power'])
plt.scatter(df_filt.loc[~df_filt['Turbine Operating'],ws_col],df_filt.loc[~df_filt['Turbine Operating'],'Active Power'])
# plt.scatter(df_filt.loc[~df_filt['Turbine Operating'],'NacelleWindSpeed'],df_filt.loc[~df_filt['Turbine Operating'],'LSS RPM'])
# plt.scatter(df_filt.loc[~df_filt['Turbine Operating'],ws_col],df_filt.loc[~df_filt['Turbine Operating'],'LSS RPM'])


In [ ]:
plt.figure()
# plt.plot(df['Yaw_Offset_Cmd_Bin'])
# plt.plot(df[ws_col])
for k in [3,4,9]:
    plt.plot(df_filt[cols2check[k]])
# plt.plot(df['Active Power'])
plt.plot(df_filt['Yaw_Offset_Cmd_Bin'])
# plt.plot(df['Yaw_Offset_Cmd'])
# plt.plot(df['WD_Nacelle'])
# plt.plot(df['Yaw_Encoder'],label='Yaw Encoder')
# plt.plot(df['Yaw_Position_Normal'],label='Yaw_Position_Normal')
# # plt.plot(df['WD1_87m'],label='WD1_87m')
# plt.plot(df['Wind_Direction_38m'],label='Wind_Direction_38m')
# plt.legend()